In [6]:
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.spatial.distance import mahalanobis
from scipy.linalg import inv
from pulp import LpMinimize, LpProblem, LpVariable, lpSum

# Step 1: Defining the Matching Framework
# Generate synthetic data for patients with repeated symptom measurements
data = pd.DataFrame({
    'id': range(1, 21),
    'urgency_baseline': np.random.randint(0, 10, 20),
    'urgency_t': np.random.randint(0, 10, 20),
    'frequency_baseline': np.random.randint(0, 10, 20),
    'frequency_t': np.random.randint(0, 10, 20),
    'pain_baseline': np.random.randint(0, 10, 20),
    'pain_t': np.random.randint(0, 10, 20),
    'age': np.random.randint(20, 80, 20),
    'gender': np.random.choice([0, 1], 20),  # 0 = Female, 1 = Male
    'treated': np.random.choice([0, 1], 20)  # 1 = treated, 0 = control
})

# Step 2: Risk Set Matching
# Separate treated and not-yet-treated patients
treated = data[data['treated'] == 1].reset_index(drop=True)
control = data[data['treated'] == 0].reset_index(drop=True)

# Compute covariance matrix and inverse for Mahalanobis distance
cov_matrix = np.cov(data[['urgency_baseline', 'urgency_t', 'frequency_baseline', 'frequency_t', 'pain_baseline', 'pain_t']].T)
cov_inv = inv(cov_matrix)

def compute_distance_matrix(treated, control):
    """Computes Mahalanobis distance matrix between treated and control groups."""
    distances = {}
    for i, t in treated.iterrows():
        for j, c in control.iterrows():
            dist = mahalanobis(t[['urgency_baseline', 'urgency_t', 'frequency_baseline', 'frequency_t', 'pain_baseline', 'pain_t']], 
                               c[['urgency_baseline', 'urgency_t', 'frequency_baseline', 'frequency_t', 'pain_baseline', 'pain_t']], 
                               cov_inv)
            distances[(i, j)] = dist
    return distances

# Compute distance matrix
distance_matrix = compute_distance_matrix(treated, control)

# Step 3: Optimal Matching via Integer Programming
prob = LpProblem("Balanced_Matching", LpMinimize)

# Define decision variables
x = {pair: LpVariable(f"x_{pair[0]}_{pair[1]}", cat='Binary') for pair in distance_matrix.keys()}

# Objective function: Minimize total distance
prob += lpSum(distance_matrix[pair] * x[pair] for pair in distance_matrix.keys())

# Constraints: Each treated patient is matched to one control
for i in range(len(treated)):
    prob += lpSum(x[(i, j)] for j in range(len(control)) if (i, j) in x) == 1

# Each control patient is matched at most once
for j in range(len(control)):
    prob += lpSum(x[(i, j)] for i in range(len(treated)) if (i, j) in x) <= 1

# Step 4: Balancing Covariates
# Enforce balance constraints on binary variables (age, gender)
for attr in ['age', 'gender']:
    treated_sum = lpSum(treated.loc[i, attr] * x[(i, j)] for i, j in x.keys())
    control_sum = lpSum(control.loc[j, attr] * x[(i, j)] for i, j in x.keys())
    prob += treated_sum == control_sum

# Step 5: Ensuring Stability and Sensitivity Analysis
# Solve the optimization problem
prob.solve()

# Extract matched pairs
matches = [(i, j) for (i, j) in x.keys() if x[(i, j)].value() == 1]
print("Matched Pairs:", matches)

# Step 6: Visualizing the Matching Process
def plot_matching_graph(treated, control, matches):
    G = nx.Graph()
    
    for i in range(len(treated)):
        G.add_node(f"T_{i}", color='red')
    for j in range(len(control)):
        G.add_node(f"C_{j}", color='blue')
    
    for i, j in matches:
        G.add_edge(f"T_{i}", f"C_{j}")
    
    colors = [G.nodes[n]['color'] for n in G.nodes]
    plt.figure(figsize=(8, 6))
    nx.draw(G, with_labels=True, node_color=colors, edge_color='black')
    plt.title("Patient Matching Graph")
    plt.show()

plot_matching_graph(treated, control, matches)

# Step 7: Box Plot Comparison
def plot_boxplots(data):
    plt.figure(figsize=(12, 6))
    melted_data = data.melt(id_vars=['treated'], value_vars=['urgency_baseline', 'urgency_t', 'frequency_baseline', 'frequency_t', 'pain_baseline', 'pain_t'], var_name='Symptom', value_name='Score')
    sns.boxplot(x='Symptom', y='Score', hue='treated', data=melted_data)
    plt.title("Comparison of Treated vs Control Groups Across Time Points")
    plt.xticks(rotation=45)
    plt.legend(title='Group', labels=['Control', 'Treated'])
    plt.show()

plot_boxplots(data)

ModuleNotFoundError: No module named 'seaborn'

In [4]:
!pip install seaborn


  Obtaining dependency information for seaborn from https://files.pythonhosted.org/packages/83/11/00d3c3dfc25ad54e731d91449895a79e4bf2384dc3ac01809010ba88f6d5/seaborn-0.13.2-py3-none-any.whl.metadata
  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
  Obtaining dependency information for pandas>=1.2 from https://files.pythonhosted.org/packages/29/d4/1244ab8edf173a10fd601f7e13b9566c1b525c4f365d6bee918e68381889/pandas-2.2.3-cp312-cp312-win_amd64.whl.metadata
  Using cached pandas-2.2.3-cp312-cp312-win_amd64.whl.metadata (19 kB)
  Obtaining dependency information for matplotlib!=3.6.1,>=3.4 from https://files.pythonhosted.org/packages/9f/6e/264673e64001b99d747aff5a288eca82826c024437a3694e19aed1decf46/matplotlib-3.10.0-cp312-cp312-win_amd64.whl.metadata
  Using cached matplotlib-3.10.0-cp312-cp312-win_amd64.whl.metadata (11 kB)
  Obtaining dependency information for contourpy>=1.0.1 from https://files.pythonhosted.org/packages/a1/35/c2de8823211d07e8a79ab018ef03960716c5dff6f4

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Python312\\share'
Consider using the `--user` option or check the permissions.


[notice] A new release of pip is available: 23.2.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip
